In [12]:
%matplotlib inline

from importlib import reload
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.stats import invwishart, invgamma, norm

# Get the macro dataset
nile = pd.read_csv('nile.csv')

In [15]:
"""
Univariate Local Linear Trend Model
"""
class LocalLinearTrend(sm.tsa.statespace.MLEModel):
    def __init__(self, endog):
        # Model order
        k_states = k_posdef = 2

        # Initialize the statespace
        super(LocalLinearTrend, self).__init__(
            endog, k_states=k_states, k_posdef=k_posdef,
            initialization='approximate_diffuse',
            loglikelihood_burn=k_states
        )

        # Initialize the matrices
        self.ssm['design'] = np.array([1, 0])
        self.ssm['transition'] = np.array([[1, 1],
                                       [0, 1]])
        self.ssm['selection'] = np.eye(k_states)

        # Cache some indices
        self._state_cov_idx = ('state_cov',) + np.diag_indices(k_posdef)

    @property
    def param_names(self):
        return ['sigma2.measurement', 'sigma2.level', 'sigma2.trend']

    @property
    def start_params(self):
        return [np.std(self.endog)]*3

    def transform_params(self, unconstrained):
        return unconstrained**2

    def untransform_params(self, constrained):
        return constrained**0.5

    def update(self, params, *args, **kwargs):
        params = super(LocalLinearTrend, self).update(params, *args, **kwargs)

        # Observation covariance
        self.ssm['obs_cov',0,0] = params[0]

        # State covariance
        self.ssm[self._state_cov_idx] = params[1:]


In [18]:
"""
Univariate Local Linear Level Model
"""
class LocalLinearLevel(sm.tsa.statespace.MLEModel):
    def __init__(self, endog):
        # Model order
        k_states = k_posdef = 1

        # Initialize the statespace
        super(LocalLinearLevel, self).__init__(
            endog, k_states=k_states, k_posdef=k_posdef,
            initialization='approximate_diffuse',
            loglikelihood_burn=k_states
        )

        # Initialize the matrices
        self.ssm['design'] = np.array([1])
        self.ssm['transition'] = np.array([[1]])
        self.ssm['selection'] = np.eye(k_states)

        # Cache some indices
        self._state_cov_idx = ('state_cov',) + np.diag_indices(k_posdef)

    @property
    def param_names(self):
        return ['sigma2.measurement', 'sigma2.level']

    @property
    def start_params(self):
        return [np.std(self.endog)]*2

    def transform_params(self, unconstrained):
        return unconstrained**2

    def untransform_params(self, constrained):
        return constrained**0.5

    def update(self, params, *args, **kwargs):
        params = super(LocalLinearLevel, self).update(params, *args, **kwargs)

        # Observation covariance
        self.ssm['obs_cov',0,0] = params[0]

        # State covariance
        self.ssm[self._state_cov_idx] = params[1:]

In [24]:
mod = LocalLinearLevel(nile['flow'])
initial_state = np.array([1000])  # Example: Modify based on your model's state dimension
initial_state_cov = np.eye(1) * 1000**2  # Small non-zero covariance for each state

# Initialize the model with known values
mod.initialize_known(initial_state, initial_state_cov)

constraints = {'sigma2.measurement': 100**2, 'sigma2.level': 100**2}
# Fit the model with constraints
res = mod.fit_constrained(constraints)

#res = mod.fit(disp=False)
print(res.summary())

                           Statespace Model Results                           
Dep. Variable:                   flow   No. Observations:                  100
Model:               LocalLinearLevel   Log Likelihood                -636.763
Date:                Thu, 29 Feb 2024   AIC                           1273.526
Time:                        18:47:39   BIC                           1273.526
Sample:                             0   HQIC                          1273.526
                                - 100                                         
Covariance Type:                  opg                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
sigma2.measurement (fixed)      1e+04        nan        nan        nan         nan         nan
sigma2.level (fixed)            1e+04        nan        nan        nan         nan 

In [19]:
mod = LocalLinearLevel(nile['flow'])

res = mod.fit(disp=False)
print(res.summary())

                           Statespace Model Results                           
Dep. Variable:                   flow   No. Observations:                  100
Model:               LocalLinearLevel   Log Likelihood                -632.538
Date:                Thu, 29 Feb 2024   AIC                           1269.075
Time:                        18:33:54   BIC                           1274.266
Sample:                             0   HQIC                          1271.175
                                - 100                                         
Covariance Type:                  opg                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
sigma2.measurement  1.513e+04   2591.445      5.837      0.000       1e+04    2.02e+04
sigma2.level        1461.9955    843.753      1.733      0.083    -191.730    3115.721
Ljung-Box (L1) (Q): 

In [13]:
# Construct a local level model for inflation
mod = sm.tsa.UnobservedComponents(nile.flow, 'llevel')

# Fit the model's parameters (sigma2_varepsilon and sigma2_eta)
# via maximum likelihood
res = mod.fit()
print(res.params)

# Create simulation smoother objects
sim_kfs = mod.simulation_smoother()              # default method is KFS
sim_cfa = mod.simulation_smoother(method='cfa')  # can specify CFA method

sigma2.irregular    15078.011658
sigma2.level         1478.811445
dtype: float64


In [4]:
nsimulations = 20
simulated_state_kfs = pd.DataFrame(
    np.zeros((mod.nobs, nsimulations)), index=dta.index)
simulated_state_cfa = pd.DataFrame(
    np.zeros((mod.nobs, nsimulations)), index=dta.index)

for i in range(nsimulations):
    # Apply KFS simulation smoothing
    sim_kfs.simulate()
    # Save the KFS simulated state
    simulated_state_kfs.iloc[:, i] = sim_kfs.simulated_state[0]

    # Apply CFA simulation smoothing
    sim_cfa.simulate()
    # Save the CFA simulated state
    simulated_state_cfa.iloc[:, i] = sim_cfa.simulated_state[0]

In [14]:
mod